In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import lightgbm as lgb
# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestRegressor,  
                               ExtraTreesRegressor)
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [2]:
%%time
df_train = pd.read_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/ultimate2/train_test/train_features.csv")
df_test = pd.read_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/ultimate2/train_test/test_features.csv")

Wall time: 8.8 s


In [4]:
print(df_train.shape)
df_test.shape

(201917, 86)


(123623, 84)

In [5]:
def feature_concat(train_data1,filepath):
    for file in filepath:
        data_features = pd.read_csv(file)
        data_features = data_features.drop_duplicates()
        train_data1=pd.merge(train_data1,data_features,how='left',on='card_id')
#         print(file)
#         print(train_data1.shape)
    return(train_data1)

In [7]:
%%time
import glob
file_list = glob.glob("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/features_90/features_80/*.csv")
train_features=feature_concat(df_train,file_list)
test_features=feature_concat(df_test,file_list)
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_features.shape

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\magic.py:187: DtypeWarning: Columns (1,9,17,28,37,44,55,64,70,78,87,94,104,113,119,131) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)


Wall time: 3min 24s


In [8]:
print(train_features.shape)
test_features.shape

(201917, 534)


(123623, 532)

In [9]:
target = df_train['target']
train_features=train_features.fillna(0)
test_features=test_features.fillna(0)

In [10]:
#Some useful parameters which will come in handy later on
ntrain = train_features.shape[0]
ntest = test_features.shape[0]
SEED = 4590 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)


# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)
    
    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [11]:
def get_oof(clf, df_train, y_train, x_test,df_train_columns):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    val_metrics=[]
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
        print("fold {}".format(fold_))
        x_tr = df_train.iloc[trn_idx][df_train_columns]
        y_tr = target.iloc[trn_idx]
        x_va = df_train.iloc[val_idx][df_train_columns]
        y_va = target.iloc[val_idx]
        clf.train(x_tr, y_tr)
        oof_train[val_idx] = clf.predict(x_va)
        oof_test_skf[fold_, :] = clf.predict(x_test[df_train_columns])
        val_metrics.append(np.sqrt(mean_squared_error(clf.predict(x_va), y_va)))
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1),val_metrics
        


In [31]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_depth': 60,
    'min_samples_leaf': 177,
    'verbose': 1
}

lgb_param1 = {
    'max_leaves':10,
             'min_data_in_leaf': 177, 
             'max_depth': 60,
            'feature_fraction':0.8,
             'objective':'regression',
             'learning_rate': 0.01,
             "boosting": "gbdt",
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": -1,
             "random_state": 4590}

lgb_param2=lgb_param1.copy()
lgb_param2['type']='gamma'

lgb_param3=lgb_param1.copy()
lgb_param3['type']='tweedie'



# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators': 10,
    'max_depth': 60,
    'min_samples_leaf' : 177,
    'verbose': 0
}

In [32]:
# Create object
rf = SklearnHelper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesRegressor, seed=SEED, params=et_params)

In [33]:
### Lightgbm
def run_lgb1(df_train,df_test,df_train_columns,lgb_params):
    param =  lgb_params
    val_metrics=[]
    oof_test_skf = np.empty((5, len(df_test)))
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
    oof_train = np.zeros(len(df_train))
    oof_test = np.zeros(len(df_test))
    feature_importance_df = pd.DataFrame()
    print(df_train.shape)
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)
#         print(trn_data.shape,val_data.shape)
        num_round = 3000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
        oof_train[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
#
        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = df_train_columns
        fold_importance_df["importance"] = clf.feature_importance()
        
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        test_predictions = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)
        oof_test_skf[fold_, :] = test_predictions
        
        predictions= clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
        val_metrics.append(np.sqrt(mean_squared_error(predictions, target.iloc[val_idx])))
    oof_test[:] = oof_test_skf.mean(axis=0)
    return feature_importance_df,oof_train.reshape(-1,1),oof_test.reshape(-1,1),val_metrics

In [34]:
### Lightgbm
def run_lgb2(df_train, df_test, lgb_params):
   param =  lgb_params
   val_metrics=[]
   oof_test_skf = np.empty((5, len(df_test)))
   folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
   oof_train = np.zeros(len(df_train))
   oof_test = np.zeros(len(df_test))
   #feature_importance_df = pd.DataFrame()
   print(df_train.shape)
   for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,train_features['outliers'].values)):
       print("fold {}".format(fold_))
       trn_data = lgb.Dataset(df_train.iloc[trn_idx], label=y_train.iloc[trn_idx])#, categorical_feature=categorical_feats)
       val_data = lgb.Dataset(df_train.iloc[val_idx], label=y_train.iloc[val_idx])#, categorical_feature=categorical_feats)
#         print(trn_data.shape,val_data.shape)
       num_round = 3000
       clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
       oof_train[val_idx] = clf.predict(df_train.iloc[val_idx], num_iteration=clf.best_iteration)
#
#        fold_importance_df = pd.DataFrame()
#        fold_importance_df["Feature"] = df_train_columns
       #fold_importance_df["importance"] = clf.feature_importance()
       
       #fold_importance_df["fold"] = fold_ + 1
       #feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

       test_predictions = clf.predict(df_test, num_iteration=clf.best_iteration)
       oof_test_skf[fold_, :] = test_predictions
       
       predictions= clf.predict(df_train.iloc[val_idx], num_iteration=clf.best_iteration)
       val_metrics.append(np.sqrt(mean_squared_error(predictions, y_train.iloc[val_idx])))
   oof_test[:] = oof_test_skf.mean(axis=0)
   return oof_train.reshape(-1,1),oof_test.reshape(-1,1),val_metrics
# stack_train=pd.DataFrame(x_train)
# target=train_features['outliers']

## Model Run

In [35]:
# # important Features
#step1:
train_features = train_features.select_dtypes(include=numerics)
train_features.columns = [x.replace("<lambda>","range") for x in train_features.columns]
test_features.columns = [x.replace("<lambda>","range") for x in test_features.columns]
df_train_columns = [c for c in train_features.columns if c not in ['card_id', 'first_active_month','target','outliers','class_outcome']]

# step2:
# df_train_columns = [c for c in train_features.columns if c not in ['card_id', 'first_active_month','target','outliers']]


# step3:
# df_train_columns = imp_features1.copy()

# # step4:
# df_train_columns.extend(['class_outcome'])


In [36]:
%%time
et_oof_train, et_oof_test,val_metrics_et = get_oof(et, train_features, target, test_features,df_train_columns) # Extra Trees
rf_oof_train, rf_oof_test,val_metrics_rf = get_oof(rf, train_features, target, test_features,df_train_columns)# Random Forest
imp_features,lgb_oof_train, lgb_oof_test,val_metrics_gb=run_lgb1(train_features,test_features,df_train_columns, lgb_param1)


fold 0
fold 1
fold 2
fold 3
fold 4
fold 0


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 21.8min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.3s finished


fold 1


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.5min


KeyboardInterrupt: 

In [30]:
print(val_metrics_et)
print(val_metrics_rf)
print(val_metrics_gb)

[3.672234580848785, 3.6717076618867504, 3.6797948623661845, 3.6797700936472086, 3.6680317694970435]


NameError: name 'val_metrics_rf' is not defined

In [ ]:
imp_features1 = imp_features.groupby('Feature').agg({'importance':sum}).reset_index().sort_values(by = 'importance', ascending = False).head(250)['Feature'].tolist()

In [ ]:
x_train = np.concatenate((et_oof_train, rf_oof_train,lgb_oof_train), axis=1)
y_train = target
x_test = np.concatenate((et_oof_test, rf_oof_test,lgb_oof_test), axis=1)

In [ ]:
stack_train=pd.DataFrame(x_train)
# target=train_features['outliers']
lgb_oof_train, lgb_oof_test,val_metrics=run_lgb2(stack_train, pd.DataFrame(x_test), lgb_param1)

In [ ]:
np.mean(val_metrics)

In [ ]:
submission=pd.DataFrame(lgb_oof_test)
submission['card_id']=test_features['card_id']
submission.columns=['target','card_id']
submission=submission[['card_id','target']]
submission.to_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/submission26th/tf_combo3.csv",index=False)
# test_class.to_csv("../fresh_code/ultimate2/features/class/test_class.csv",index=False)

In [ ]:
submission.shape

In [ ]:
stack_train['target'] = y_train
stack_train['card_id'] = df_train['card_id']
stack_test = pd.DataFrame(x_test)
stack_test['card_id'] = test_features['card_id']
stack_train.to_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/stack_train/tfs_combo1.csv",index=False)
stack_test.to_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/stack_test/tfs_combo1.csv",index=False)